# Develop, Train, Optimize, and Deploy Scikit-Learn LinearSVC
## Twitter Sentiment

In [8]:
import boto3
import json
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sagemaker.sklearn.estimator import SKLearn

# SageMaker Python SDK
sm_boto3 = boto3.client('sagemaker')
sess = sagemaker.Session()
region = sess.boto_session.region_name

bucket = sess.default_bucket()
print('Using bucket ' + bucket)


Using bucket sagemaker-us-east-1-159307201141


### Prepare data

In [ ]:
DATASET_COLUMNS = ['target', 'ids', 'date', 'flag', 'user', 'tweet']
DATASET_ENCODING = 'ISO-8859-1'
TRAIN_SIZE = 0.8

# Read the data locally
df = pd.read_csv('svc/training.1600000.processed.noemoticon.csv',
                 encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

# Split the data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    df['tweet'], df['target'], test_size=1 - TRAIN_SIZE, random_state=817)

trainX = pd.DataFrame(X_train, columns=['tweet'])
trainX['target'] = y_train

testX = pd.DataFrame(X_test, columns=['tweet'])
testX['target'] = y_test

# Save the train_test_split locally
trainX.to_csv('svc/twitter_train.csv', index=False)
testX.to_csv('svc/twitter_test.csv', index=False)

# Send data to S3. SageMaker will take training data from S3
trainpath = sess.upload_data(
    path='svc/twitter_train.csv', bucket=bucket,
    key_prefix='data/twitter')

testpath = sess.upload_data(
    path='svc/twitter_test.csv', bucket=bucket,
    key_prefix='data/twitter')


### Writing a Script Mode Script

In [ ]:
%%writefile script.py

import argparse
import json
import os
import re
from io import StringIO
from distutils.util import strtobool

import numpy as np
import pandas as pd

from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sagemaker_containers.beta.framework import worker

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

TWEET_CLEANING_RE = r'@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+'

decode_map = {
    0: 'NEGATIVE',
    2: 'NEUTRAL',
    4: 'POSITIVE'
}


# TODO Do I need this?
def decode_sentiment(label):
    return decode_map[int(label)]


# TODO Adjust preprocessor to be better
def preprocess(tweet, stem=False):
    """Preprocesses one tweet"""
    tweet = re.sub(TWEET_CLEANING_RE, ' ', str(tweet).lower()).strip()
    tokens = []
    for token in tweet.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)


def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return clf


def input_fn(request_body, request_content_type):
    """An input_fun that loads JSON into a Pandas DataFrame, and preprocesses the tweets"""
    if request_content_type == "application/json":
        # TODO We don't really need Pandas here anymore
        df = pd.read_json(StringIO(request_body))
        # TODO Add way to provide stem parameter
        df.tweet = df.tweet.apply(lambda x: preprocess(x))
        return df['tweet'].to_numpy()
    else:
        raise ValueError(
            '{} is not supported by script.'.format(request_content_type))


def predict_fn(input_data, model):
    pred = model.predict(input_data)
    deci_func = model.decision_function(input_data)

    predictions = []
    for p, d in zip(pred, deci_func):
        predictions.append({
            'prediction': p,
            'probability': d
        })

    return {'results': predictions}


def output_fn(prediction, content_type):
    if content_type == 'application/json':
        return worker.Response(json.dumps(prediction), content_type, mimetype=content_type)
    else:
        raise ValueError(
            '{} accept type is not supported by this script.'.format(content_type))


if __name__ == '__main__':
    print('Extracting arguments')

    parser = argparse.ArgumentParser()

    # Hyperparameters from the client
    parser.add_argument('--stem', type=bool, default=False)
    parser.add_argument('--use-idf', type=str, default='true')
    parser.add_argument('--smooth-idf', type=str, default='true')
    parser.add_argument('--sublinear-tf', type=str, default='true')
    parser.add_argument('--C', type=float, default=1.0)
    parser.add_argument('--penalty', type=str, default='l2')
    parser.add_argument('--loss', type=str, default='squared_hinge')

    # Data, model, and output directories
    # TODO Remove this argument?
    # parser.add_argument('--output-data-dir', type=str,
    #                     default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model-dir', type=str,
                        default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str,
                        default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str,
                        default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='twitter_train.csv')
    parser.add_argument('--test-file', type=str, default='twitter_test.csv')

    args, _ = parser.parse_known_args()

    print('Reading Tweets')
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print('Length of train_df: {}'.format(str(len(train_df.index))))
    print('Length of test_df: {}'.format(str(len(test_df.index))))

    print('Preprocessing the Tweets')
    # Decode Sentiment/Target
    train_df.target = train_df.target.apply(lambda x: decode_sentiment(x))
    test_df.target = test_df.target.apply(lambda x: decode_sentiment(x))

    # Preprocess Tweet
    train_df.tweet = train_df.tweet.apply(lambda x: preprocess(x, args.stem))
    test_df.tweet = test_df.tweet.apply(lambda x: preprocess(x, args.stem))

    print('Building training and testing datasets')
    X_train = train_df['tweet']
    y_train = train_df['target']
    X_test = test_df['tweet']
    y_test = test_df['target']

    # TODO TfIdf preprocessor?

    print('Training the model')
    pipe = Pipeline([
        ('tfidf', TfidfVectorizer(use_idf=bool(strtobool(args.use_idf)),
                                  smooth_idf=bool(strtobool(args.smooth_idf)),
                                  sublinear_tf=bool(strtobool(args.sublinear_tf)))),
        ('svc', LinearSVC(C=args.C,
                          penalty=args.penalty,
                          loss=args.loss))
    ])

    clf = pipe.fit(X_train, y_train)

    print('Print validation statistics')
    pred = clf.predict(X_test)
    # pred_prob = clf.predict_proba(X_test)
    decision = clf.decision_function(X_test)

    print(pred)
    print(decision)

    # TODO Why the fuck are these the same?
    print('Accuracy: {}'.format(accuracy_score(y_test, pred)))
    # print('Precision: {}'.format(precision_score(y_test, pred, average='macro')))
    # print('Recall: {}'.format(recall_score(y_test, pred, average='micro')))
    # TODO Add more validation statistics

    print('Save the model')
    joblib.dump(clf, os.path.join(args.model_dir, 'model.joblib'))

### Local training

In [ ]:
! python svc/script.py -C 6.8437521959309535 \
                    --smooth_idf false \
                    --sublinear_tf false \
                    -- use_idf true \
                    --model-dir svc/ \
                    --train svc/ \
                    --test svc/

### SageMaker Training

In [ ]:
# We use the Estimator from the SageMaker Python SDK

sklearn_estimator = SKLearn(
    entry_point='svc.py',
    source_dir='svc/',
    role = get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.m5.large',
    framework_version='0.20.0',
    base_job_name='twitter-svc',
    metric_definitions=[
        {'Name': 'accuracy',
         'Regex': 'Accuracy: ([0-9.]+).*$'}],
    hyperparameters={
        'C': 6.8437521959309535,
        'smooth_idf': 'false',
        'sublinear_tf': 'true',
        'use_idf': 'true'
    }
)

In [ ]:
sklearn_estimator.fit({'train':trainpath, 'test':testpath})

### Launching a tuning job with the Python SDK

In [ ]:
# We use the Hyperparameter Tuner 

from sagemaker.tuner import ContinuousParameter, CategoricalParameter

hyperparameter_ranges = {
    'use_idf': CategoricalParameter(['true', 'false']),
    'smooth_idf': CategoricalParameter(['true', 'false']),
    'sublinear_tf': CategoricalParameter(['true', 'false']),
    'C': ContinuousParameter(0.0001, 100, 'Logarithmic')
    # 'penalty': CategoricalParameter(['l1', 'l2']),
    # 'loss': CategoricalParameter(['hinge', 'squared_hinge'])
}

Optimizer = sagemaker.tuner.HyperparameterTuner(
    estimator=sklearn_estimator,
    hyperparameter_ranges=hyperparameter_ranges,
    base_tuning_job_name='twitter-svc-tuner',
    objective_type='Maximize',
    objective_metric_name='accuracy',
    metric_definitions=[
        {'Name': 'accuracy',
         'Regex': 'Accuracy: ([0-9.]+).*$'}
    ],
    max_jobs=20,
    max_parallel_jobs=3
)

In [ ]:
Optimizer.fit({'train':trainpath, 'test':testpath})

In [ ]:

results = Optimizer.analytics().dataframe()
results.head()

### Deploy to a real-time endpoint

In [6]:
#TODO Get best model from a tuning job OR do it by name

my_training_job_name = "twitter-svc-2020-03-10-01-42-11-418"
sklearn_estimator = SKLearn.attach(my_training_job_name)

predictor = sklearn_estimator.deploy(
    instance_type='ml.t2.medium',
    initial_instance_count=1
)

2020-03-10 01:47:30 Starting - Preparing the instances for training
2020-03-10 01:47:30 Downloading - Downloading input data
2020-03-10 01:47:30 Training - Training image download completed. Training in progress.
2020-03-10 01:47:30 Uploading - Uploading generated training model
2020-03-10 01:47:30 Completed - Training job completed2020-03-10 01:46:40,466 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-03-10 01:46:40,468 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-03-10 01:46:40,478 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-03-10 01:46:57,241 sagemaker-containers INFO     Module svc does not provide a setup.py. 
Generating setup.py
2020-03-10 01:46:57,241 sagemaker-containers INFO     Generating setup.cfg
2020-03-10 01:46:57,242 sagemaker-containers INFO     Generating MANIFEST.in
2020-03-10 01:46:57,242 sagemaker-containers INFO     Installing module with the follow

### Invoke with boto3

In [9]:
runtime = boto3.client('sagemaker-runtime')

tweets = {
    'tweet': [
        '''totally okay to not love Biden but if you’re threatening to just not vote in the election if he’s the candidate, you suck''',
        '''Here to give you dinner inspo, so you don't keep eating chips for dinner.''',
        '''Fresh pastry from Beetbox in the house!''',
        '''Spent 2 days in New Orleans. Gained 75 lbs. Totally worth it.''',
        '''These people made me laugh so hard that I briefly thought I gave myself a hernia.'''
    ]
}

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=json.dumps(tweets),
    ContentType='application/json')

print(response['Body'].read())


b'{"results": [{"prediction": "POSITIVE", "probability": 1.2966381957664954}, {"prediction": "POSITIVE", "probability": 0.7260198933212587}, {"prediction": "NEGATIVE", "probability": -0.1334945546460503}, {"prediction": "POSITIVE", "probability": 0.8456000844949692}, {"prediction": "NEGATIVE", "probability": -0.34600364036305226}]}'


In [ ]:
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)